# DataLoader 和 Dataset

构建模型的基本方法，我们了解了。接下来，我们就要弄明白怎么对数据进行预处理，然后加载数据，我们以前手动加载数据的方式，在数据量小的时候，并没有太大问题，但是到了大数据量，我们需要使用 shuffle, 分割成mini-batch 等操作的时候，我们可以使用PyTorch的API快速地完成这些操作。

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.autograd import Variable
import numpy as np

In [8]:
xy = np.loadtxt('./dataSet/diabetes.csv.gz', delimiter=',', dtype=np.float32) # 使用numpy读取数据
x_data = torch.from_numpy(xy[:, 0:-1])
y_data = torch.from_numpy(xy[:, [-1]])

In [9]:
print(x_data.shape, y_data.shape)

torch.Size([759, 8]) torch.Size([759, 1])


Dataset是一个包装类，用来将数据包装为Dataset类，然后传入DataLoader中，我们再使用DataLoader这个类来更加快捷的对数据进行操作。

DataLoader是一个比较重要的类，它为我们提供的常用操作有：batch_size(每个batch的大小), shuffle(是否进行shuffle操作), num_workers(加载数据的时候使用几个子进程), pin_memory(将数据放在CUDA上)

现在，我们先展示直接使用 TensorDataset 来将数据包装成Dataset类

In [10]:
deal_dataset = TensorDataset(data_tensor=x_data, target_tensor=y_data)

In [11]:
train_loader = DataLoader(dataset=deal_dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [12]:
for epoch in range(2):
    for i, data in enumerate(train_loader):
        # 将数据从 train_loader 中读出来,一次读取的样本数是32个
        inputs, labels = data

        # 将这些数据转换成Variable类型
        inputs, labels = Variable(inputs), Variable(labels)

        # 接下来就是跑模型的环节了，我们这里使用print来代替
        print(epoch, i, "inputs", inputs.data.size(), "labels", labels.data.size())

0 0 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 1 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 2 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 3 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 4 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 5 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 6 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 7 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 8 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 9 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 10 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 11 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 12 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 13 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 14 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 15 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 16 inputs torch.Size([32, 8]) labels torch.Size([32, 1])
0 17 in

接下来，我们来继承 Dataset类 ，写一个处理将数据处理成DataLoader的类。

对于一个 Dataset类 我们需要重写 \_\_len\_\_ 方法，该方法提供了dataset的大小；  \_\_getitem\_\_ 方法， 该方法支持从 0 到 len(self)的索引

In [5]:
class DealDataset(Dataset):
    """
        下载数据、初始化数据，都可以在这里完成
    """
    def __init__(self):
        xy = np.loadtxt('./dataSet/diabetes.csv.gz', delimiter=',', dtype=np.float32) # 使用numpy读取数据
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
        self.len = xy.shape[0]
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

# 实例化这个类，然后我们就得到了Dataset类型的数据，记下来就将这个类传给DataLoader，就可以了。    
dealDataset = DealDataset()

train_loader2 = DataLoader(dataset=dealDataset,
                          batch_size=32,
                          shuffle=True)


for epoch in range(2):
    for i, data in enumerate(train_loader2):
        # 将数据从 train_loader 中读出来,一次读取的样本数是32个
        inputs, labels = data

        # 将这些数据转换成Variable类型
        inputs, labels = Variable(inputs), Variable(labels)

        # 接下来就是跑模型的环节了，我们这里使用print来代替
        print("epoch：", epoch, "的第" , i, "个inputs", inputs.data.size(), "labels", labels.data.size())

epoch： 0 的第 0 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 1 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 2 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 3 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 4 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 5 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 6 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 7 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 8 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 9 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 10 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 11 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 12 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 13 个inputs torch.Size([32, 8]) labels torch.Size([32, 1])
epoch： 0 的第 14 个inputs torch.S

# torchvision 包的介绍

